In [2]:
import os
import boto3

# Set the AWS profile
os.environ['AWS_PROFILE'] = 'ccbd-joseph'

# Now you can use boto3 to interact with AWS services
# For example, creating an S3 client
s3 = boto3.client('s3')

# List buckets
response = s3.list_buckets()
for bucket in response['Buckets']:
    print(f'Bucket Name: {bucket["Name"]}')


Bucket Name: cf-templates-1g550bcpoyriy-us-east-1
Bucket Name: cf-templates-1g550bcpoyriy-us-east-2
Bucket Name: dinnerconcierge-jg4376
Bucket Name: photo-album-assets
Bucket Name: photos-frontend-jg4376
Bucket Name: photos-jg4376
Bucket Name: sagemaker-studio-561013435358-e1wss47m5oa
Bucket Name: sagemaker-studio-561013435358-ezvq1b3ry
Bucket Name: sagemaker-us-east-1-561013435358
Bucket Name: skinfix-frontend
Bucket Name: skinfix-ml-bucket
Bucket Name: skinfix-patient-photos
Bucket Name: skinfix-train-data
Bucket Name: voice-files-all


In [5]:
import boto3
from opensearchpy import OpenSearch, helpers


# Initialize clients for DynamoDB and OpenSearch
dynamodb = boto3.resource('dynamodb')
opensearch = OpenSearch(
    hosts=[{'host': 'search-doctors-domain-jhukjvozwcmrtu4a3ngi6umdc4.us-east-1.es.amazonaws.com', 'port': 443}],
    use_ssl=True,
    verify_certs=True
)


# DynamoDB table
table = dynamodb.Table('doctorsInfo')

# Scan DynamoDB table (consider pagination for large tables)
response = table.scan()
items = response['Items']

# Process records and create mapping
focus_mapping = {}
for item in items:
    doctor_id = item['doctorId']
    focus_areas = item['focus']
    for focus in focus_areas:
        if focus not in focus_mapping:
            focus_mapping[focus] = []
        focus_mapping[focus].append(doctor_id)
# Prepare data for OpenSearch indexing
actions = [
    {
        "_index": "doctorFocusAreas",
        "_id": focus,
        "_source": {
            "focus_area": focus,
            "doctor_ids": list(set(focus_mapping[focus]))
        }
    }
    for focus in focus_mapping
]

# Bulk index data into OpenSearch
helpers.bulk(opensearch, actions)


RequestError: RequestError(400, '<html>\r\n<head><title>400 The plain HTTP request was sent to HTTPS port</title></head>\r\n<body>\r\n<center><h1>400 Bad Request</h1></center>\r\n<center>The plain HTTP request was sent to HTTPS port</center>\r\n</body>\r\n</html>\r\n')